# Eager Execution Basics

In [1]:
import tensorflow as tf
import gc

tf.enable_eager_execution()

In [6]:
print(tf.add(1,2))
print(tf.add([1,2], [3,4]))
print(tf.square(5))
print(tf.reduce_sum([1,2,3]))
print(tf.encode_base64("hello world"))

print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


In [9]:
x = tf.matmul([[1]], [[2,3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


Converting between Numpy ndarrays and Tensorflow Tensors

In [16]:
gc.collect()

import numpy as np

ndarray = np.ones([3, 3])

print("Tensorflow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)

print("And NumPy operations can convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method excplicitly converts a Tensor to a numpy array")
print(tensor.numpy())


Tensorflow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations can convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method excplicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


GPU acceleration

In [23]:
x = tf.random_uniform([3,3])

print(f"Is there a gpu avalaible: {tf.test.is_gpu_available()}")
print(f"is the Tensor on gpu #0: {x.device.endswith('GPU:0')}")

Is there a gpu avalaible: True
is the Tensor on gpu #0: True


In [26]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [27]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x,x)
    
    result = time.time()-start
    
    print("10 loops: {:0.2f}ms".format(1000*result))

# Execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random_uniform([1000,1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# Execition on GPU
print("on GPU:")
with tf.device("GPU:0"):
    x = tf.random_uniform([1000,1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 96.00ms
on GPU:
10 loops: 251.28ms


Creating Datasets

In [28]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
    Line2
    Line3
      """)
    
ds_file = tf.data.TextLineDataset(filename)

In [29]:
# Apply some transfomrations
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [34]:
# Iterate
print("Elements of ds_tensors:")
for x in ds_tensors:
    print(x)
    
print("\nElements in ds_file:")
for x in ds_file:
    print(x)

Elements of ds_tensors:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([36  9], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'    Line2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line3' b'      '], shape=(2,), dtype=string)
